# ***Disclaimer:*** 
Hello Kagglers! I am a Solution Architect with the Google Cloud Platform. I am a coach for this competition, the focus of my contributions is on helping users to leverage GCP components (GCS, TPUs, BigQueryetc..) in order to solve large problems. My ideas and contributions represent my own opinion, and are not representative of an official recommendation by Google. Also, I try to develop notebooks quickly in order to help users early in competitions. There may be better ways to solving particular problems, I welcome comments and suggestions. Use my contributions at your own risk, I don't garantee that they will help on winning any competition, but I am hoping to learn by collaborating with everyone.

# Objective:


The objective of this notebook is to demonstrate how to train a model using Keras.fit using accelerators (GPU or TPU). 

The Keras model utilized is the one proposed by a [popular paper in biomedical image segmentation](https://arxiv.org/abs/1505.04597), by (Olaf Ronneberger, Philipp Fischer, Thomas Brox).

The particular implementation used is the one proposed by by [Dr. Bradley Erickson](https://github.com/slowvak), available in the: [The Magician's Corner repository](https://github.com/RSNA/MagiciansCorner/blob/master/UNetWithTensorflow.ipynb). 

The basic modification that I have made to the implementation provided by Dr. Erickson is to enable the Tensorflow distributed training strategy (tf.strategy). You will notice that the function model.fit() is used within a strategy.scope(), so that it leverages either GPU or TPU acceleration. 

Note: The TPU implementation is currently running slow because the TFRecord file size is very small, resulting in thousand of files that need to be open. In the future I will provide a dataset that is more appropriate for TPUs. I recommend using GPUs for now.

In previous notebooks, I demonstrated how to read the competition data and produce a TFRecord dataset tiling the images in 512x512 tiles. This Notebook will use this dataset as input to the Keras Unet model:
--> [Link to the TFRecord Dataset Used by this Notebook.](https://www.kaggle.com/marcosnovaes/hubmap-tfrecord-512)

Previous Notebooks in this competition: 

[https://www.kaggle.com/marcosnovaes/hubmap-3-unet-models-with-keras-cpu-gpu/](https://www.kaggle.com/marcosnovaes/hubmap-3-unet-models-with-keras-cpu-gpu/): Investigates three implementations of the Unet model

[https://www.kaggle.com/marcosnovaes/hubmap-read-data-and-build-tfrecords/](https://www.kaggle.com/marcosnovaes/hubmap-read-data-and-build-tfrecords/): Demonstrates how the TFRecord Dataset was built

[https://www.kaggle.com/marcosnovaes/hubmap-looking-at-tfrecords/](https://www.kaggle.com/marcosnovaes/hubmap-looking-at-tfrecords/): Explains how to read the data using the TFRecord Dataset

# Setup
1) Add the TFRecord Dataset as input to the notebook: Go to the Data section at the right, click "add data" and lof for the dataset: "hubmap_train_test"

2) This Notebook also shows how to access a Kaggle dataset directly from Google Cloud Storage (GCS). To enable this feature, you need to link the Notebook to a GCS project, by going to the menu Add-ons-->Cloud SDK

In [ ]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input

from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D
from tensorflow.keras.layers import concatenate

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

from tensorflow.keras import layers

from keras.engine.topology import Layer

from tensorflow.keras.optimizers import Adam
from keras.utils.generic_utils import get_custom_objects


from kaggle_datasets import KaggleDatasets
from kaggle_secrets import UserSecretsClient

import tensorflow as tf

In [ ]:
#ACCELERATOR_TYPE = 'TPU'
ACCELERATOR_TYPE = 'GPU'

In [ ]:
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
if ACCELERATOR_TYPE == 'TPU':
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)

    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
# Create a dictionary describing the features.
image_feature_description = {
    'img_index': tf.io.FixedLenFeature([], tf.int64),
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'num_channels': tf.io.FixedLenFeature([], tf.int64),
    'img_bytes': tf.io.FixedLenFeature([], tf.string),
    'mask': tf.io.FixedLenFeature([], tf.string),
    'tile_id': tf.io.FixedLenFeature([], tf.int64),
    'tile_col_pos': tf.io.FixedLenFeature([], tf.int64),
    'tile_row_pos': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_image_and_masks_function(example_proto):
    single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    img_height = single_example['height']
    img_width = single_example['width']
    num_channels = single_example['num_channels']
    
    img_bytes =  tf.io.decode_raw(single_example['img_bytes'],out_type='uint8')
    #dynamic shape
    #img_array = tf.reshape( img_bytes, (img_height, img_width, num_channels))
    #fixed shape
    img_array = tf.reshape( img_bytes, (512, 512, 3))
    
    mask_bytes =  tf.io.decode_raw(single_example['mask'],out_type='bool')

    mask = tf.reshape(mask_bytes, (512,512))
    
    #normalize images array and cast image and mask to float32
    img_array = tf.cast(img_array, tf.float32) / 255.0
    mask = tf.cast(mask, tf.float32)
    return img_array, mask

def read_dataset(storage_file_path):
    encoded_image_dataset = tf.data.TFRecordDataset(storage_file_path, compression_type="GZIP")
    parsed_image_dataset = encoded_image_dataset.map(_parse_image_and_masks_function)
    return parsed_image_dataset

In [ ]:
with strategy.scope():
    def dice_coeff(y_true, y_pred):
        # add epsilon to avoid a divide by 0 error in case a slice has no pixels set
        # we only care about relative value, not absolute so this alteration doesn't matter
        _epsilon = 10 ** -7
        intersections = tf.reduce_sum(y_true * y_pred)
        unions = tf.reduce_sum(y_true + y_pred)
        dice_scores = (2.0 * intersections + _epsilon) / (unions + _epsilon)
        return dice_scores

    def dice_loss(y_true, y_pred):
        loss = 1 - dice_coeff(y_true, y_pred)
        return loss
  
    get_custom_objects().update({"dice": dice_loss})

    class LayerNormalization (Layer) :
    
        def call(self, x, mask=None, training=None) :
            axis = list (range (1, len (x.shape)))
            x /= K.std (x, axis = axis, keepdims = True) + K.epsilon()
            x -= K.mean (x, axis = axis, keepdims = True)
            return x
        
        def compute_output_shape(self, input_shape):
            return input_shape

In [ ]:
def magic_unet(act_fn = 'relu', init_fn = 'he_normal', width=512, height = 512, channels = 3): 
    inputs = Input((512,512,3))
    act_fn = 'relu'
    init_fn = 'he_normal'

    # note we use linear function before layer normalization
    conv1 = Conv2D(8, 5, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(inputs)
    conv1 = LayerNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(16, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(32, 3, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(pool2)
    conv3 = LayerNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(64, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(72, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(pool4)

    up6 = Conv2D(64, 2, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv5))
    up6 = LayerNormalization()(up6)
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(64, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge6)

    up7 = Conv2D(32, 2, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(32, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge7)

    up8 = Conv2D(16, 2, activation = 'linear', padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv7))
    up8 = LayerNormalization()(up8)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(16, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge8)

    up9 = Conv2D(8, 2, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(8, 3, activation = act_fn, padding = 'same', kernel_initializer = init_fn)(merge9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs = inputs, outputs = conv10)

    return model


In [ ]:
!ls /kaggle/input

In [ ]:
!ls /kaggle/input/hubmap-tfrecord-512

In [ ]:
train_tiles_csv = '/kaggle/input/hubmap-tfrecord-512/train_all_tiles.csv'
test_tiles_csv = '/kaggle/input/hubmap-tfrecord-512/test_all_tiles.csv'

In [ ]:
# build a dataset of all images tiles from the train set that have gloms in them
#for csv_file in file_list:
train_tiles_df = pd.read_csv(train_tiles_csv)
train_gloms_df = train_tiles_df.loc[train_tiles_df["mask_density"]  > 0]
train_gloms_df.head()

In [ ]:
train_gloms_df.__len__()

In [ ]:
train_cropped_df = train_tiles_df.loc[train_tiles_df["lowband_density"]  > 1000]
train_cropped_df.head()

In [ ]:
train_cropped_df.__len__()

In [ ]:
if ACCELERATOR_TYPE == 'TPU':
    train_files = train_gloms_df[0:1000]['gcs_path']
    test_files = train_gloms_df[1000:1050]['gcs_path']
else:
    train_files = train_gloms_df[0:1000]['local_path']
    test_files = train_gloms_df[1000:1050]['local_path']

train_dataset = read_dataset(train_files)
test_dataset = read_dataset(test_files)

for image, mask in train_dataset.take(1):
    train_image, train_mask = image, mask
    
for image, mask in test_dataset.take(1):
    test_image, test_mask = image, mask

fig, ax = plt.subplots(2,2,figsize=(10,6))
ax[0][0].imshow(train_image)
ax[0][1].imshow(train_mask)
ax[1][0].imshow(test_image)
ax[1][1].imshow(test_mask)


In [ ]:
with strategy.scope():
   
    model = magic_unet()
    model.compile(optimizer = Adam(lr = 1e-5), loss = 'dice', metrics=[dice_coeff])
    
    if ACCELERATOR_TYPE == 'TPU':
        train_files = train_gloms_df[0:1024]['gcs_path']
        test_files = train_gloms_df[1024:1536]['gcs_path']
    else:
        train_files = train_gloms_df[0:1000]['local_path']
        test_files = train_gloms_df[1000:1050]['local_path']

    train_dataset = read_dataset(train_files)
    train_dataset = train_dataset.batch(8, drop_remainder=True).cache()
    test_dataset = read_dataset(test_files)
    test_dataset = test_dataset.batch(8, drop_remainder=True).cache()

    batch_size = 8
    #steps_per_epoch = 60000 // batch_size
    steps_per_epoch = 100
    validation_steps = 10000 // batch_size
    
    checkpointer = ModelCheckpoint('/kaggle/working/model-hubmap.h5', verbose=1)
    model.fit(train_dataset,batch_size=8, epochs=20, validation_data=test_dataset,callbacks=[checkpointer])
    
          #steps_per_epoch=steps_per_epoch,
          #validation_data=test_dataset, 
          #validation_steps=validation_steps)
    #earlystopper = EarlyStopping(patience=5, verbose=1)
    #checkpointer = ModelCheckpoint('/kaggle/working/model-hubmap.h5', verbose=1)

    #results = unet_model.fit(train_dataset, batch_size=1, epochs=1, callbacks=[checkpointer])


In [ ]:
!ls -l /kaggle/working

In [ ]:
small_test = train_gloms_df[2000:2008]['local_path']
small_dataset = read_dataset(small_test)
small_dataset = small_dataset.batch(1, drop_remainder=True).cache()

for image, mask in small_dataset.take(1):
    test_image, test_mask = image, mask
test_image.shape

In [ ]:
plt.imshow(test_image[0,:,:,:])

In [ ]:
pred_model = magic_unet()
pred_model.load_weights("/kaggle/working/model-hubmap.h5")

In [ ]:
pred_mask = pred_model.predict(test_image, verbose=1)
pred_mask.shape

In [ ]:
pred_mask[0,:,:,0]

In [ ]:
plt.imshow(pred_mask[0,:,:,0])

In [ ]:
bool_mask = pred_mask[0,:,:,0] > 0.5
plt.imshow(bool_mask)

In [ ]:
#unet_model.load_weights("/kaggle/working/model-hubmap.h5")

test_image = []
test_mask = []
pred_mask = []
for image, mask in small_dataset.take(1):
    test_image, test_mask = image, mask
    pred_mask = pred_model.predict(test_image, verbose=1)
    bool_mask = (pred_mask > 0.5)
    
fig, ax = plt.subplots(1,3,figsize=(20,3))
ax[0].imshow(test_image[0,:,:,:])
ax[1].imshow(test_mask[0,:,:])
ax[2].imshow(bool_mask[0,:,:,0])

In [ ]:
mask_density = np.count_nonzero(pred_mask)
mask_density

In [ ]:
!ls -l /kaggle/working